### 1. SQL66 牛客每个人最近的登录日期(一)

* what I do: RANK BY DATE WITHIN PARTITION(GROUP) 在每个分组中来对注册日期 进行排序。 如果我直接在新城生的LOGIN 表格中 来用WHERE 
  会报错， 所以我选择把新生成的LOGIN 当作一张表， 然后从这张表中重新选择

In [ ]:
SELECT s.user_id, s.date FROM
(SELECT user_id, date, 
RANK() OVER(PARTITION BY user_id ORDER BY date DESC) AS rak FROM login) AS s
WHERE s.rak=1

In [ ]:
| 牛客给的解法》 使用 集合函数 与GROUP BY 在一起 这样 我们可以不用担心 必须在GROUP BY 里面选择所有SELECT 中出现的COLUMN 因为
 column 也可以出现在 集合函数中

In [ ]:
# HOW TO do filter or order by within each group? 
SELECT user_id, MAX(date) as d FROM login 
GROUP BY user_id 
ORDER BY user_id ASC

### 2. SQL67 牛客每个人最近的登录日期(二)  较难
牛客每天有很多人登录，请你统计一下牛客每个用户最近登录是哪一天，用的是什么设备.

In [ ]:
| 我的想法 拿到这个题 因为也是多表格 属于份表格的情况 那么我们就要 考虑 从小表格 开始 达成 局部目标， 得到部分想要的数据 再 合并各个
表格 可以用 笛卡尔积 where 或者 JOIN 
我是 先把 每个用户 最近登录的时间 导出做成一个表格， 用到了 RANK在 每个GROUP 里面 排序 这样可以包含 CLIENT?ID 如果用
GROUP BY 的话 那么 我们DATE 放在 MAX 函数里面 但是 CLIENT?ID 不能选 因为我们需要它来找到 设备的名字 但是它如果选了的话 那就是 GROUP
BY USER?ID 和 CLIENT?ID 不是我们想要的 我们就想 对USER?ID 单独 排除重复项
因为同一个 用户也有可能用不同设备 


In [2]:
SELECT user.name as u_n, table2.c_n, table2.date  FROM user,

(SELECT c_rank.user_id, client.name as c_n, c_rank.date FROM client,
(SELECT user_id, client_id, date, RANK() OVER(PARTITION BY user_id ORDER BY date DESC)
 AS c_ran FROM login ) AS c_rank
WHERE c_rank.c_ran=1 and client.id=c_rank.client_id) AS table2
WHERE user.id=table2.user_id 
ORDER BY u_n ASC

SyntaxError: invalid syntax (<ipython-input-2-2ed0045443e8>, line 1)

### 3. SQL68 牛客每个人最近的登录日期(三)
牛客每天有很多人登录，请你统计一下牛客新登录用户的次日成功的留存率
(sqlite里查找某一天的后一天的用法是:date(yyyy-mm-dd, '+1 day')，四舍五入的函数为round，sqlite 1/2得到的不是0.5，得到的是0，只有1*1.0/2才会得到0.5
mysql里查找某一天的后一天的用法是:DATE_ADD(yyyy-mm-dd,INTERVAL 1 DAY)，四舍五入的函数为round)

#### 就是判断他们有没有在第一登录后一天的日期登录过 这就叫次日保存率 
#### 对于每一行 我们 看他是不是第一次登录 所以我们考虑所有用户最近登录的表格
#### 他是的话 那么再对他进行后面一天的查看 但是用CASE 查看的时候我们用uni_login 因为我们不需要看同一个用户在其首次登录的那天重复#### 登录了几次 然后 在考虑 CASE 的判断条件， 对于当前行， 我们不仅要求它是首次登录 就是在首次登录的日期表中， 还要要求它在其首
#### 次登陆后的第二天也登陆了 就是它也在 它的ID 和后一天登录日期也在表格中 那么他就是存留成功客户标记为1 不然就是0 
#### 这样我们对 UNI?LOHGIN 中每一行都有了标记。 然后就是 把这个标记好的表格作为一个表 来从里面计算概率

In [ ]:

SELECT ROUND(SUM(tt.cn)/COUNT(tt.cn),3) FROM 
(SELECT uni_login.user_id,
    SUM(CASE 
    WHEN (uni_login.user_id, uni_login.date) 
    IN (SELECT login.user_id, min(login.date) FROM login GROUP BY login.user_id)
    AND (uni_login.user_id, DATE_ADD(uni_login.date,INTERVAL 1 DAY))
    IN (SELECT login.user_id, login.date FROM login GROUP BY login.user_id, login.date)
    THEN 1 ELSE 0 END ) AS cn
FROM 
(SELECT user_id, date FROM login 
GROUP BY user_id, date) AS uni_login
GROUP BY uni_login.user_id) AS tt

### 4.SQL69 牛客每个人最近的登录日期(四)
要求 求每天 新用户登录个数


In [ ]:
# group by date. within each group 
#SELECT date, COUNT(DISTINCT user_id) FROM login 
#GROUP BY date 
# 1. 分组 每个组里都有 日期和 USER_ID 
# 2.最早的日期里 每个distinct 的 都是第一次登录 之后 登录日期在之前组里出现过的 都是老用户
# 我可以想到 要用到CASE 来计数 CASE 的结果会带来新的一列
# 题解中用到 技术神器 SUM-CASE 所以CASE 会得出一列 SUM 会对这列求和
#SELECT user_id, min(date)FROM login 
#GROUP BY user_id
#2|2020-10-12 用户胡id 2 在最近登录日期是12 号
#3|2020-10-12
#1|2020-10-12
#4|2020-10-14
# 按照 用户分类 每个用户可以在不同日期登录设备 我们删选出它最近登陆的日期
# case when (user_id,date) in 
#    (select user_id,min(date)from login group by user_id)
#    then 1 else 0 end
# CASE 在这是对每一行 进行 删选 当这一行的 USER?ID DATE 是在我们最新登录记录中 那么就计数 1
# 比如 第一行中 2号用户最近登录在12 号 就计数 1 但是当我们到第二行时候 用户2 号 有登陆了 
# 但是这不是他的第一次登录 我们不计算在内 计数为0 .。。 当我们遍历过所有行数 就会得到一个
# 计数的列 全是0 或者1. 我们SUM 它 就是 原始列表 新的 列 计数的最终结果
#SELECT date, 
#CASE
#    WHEN (user_id, date) IN (SELECT user_id, min(date) FROM login GROUP BY user_id)
#    THEN 1 ELSE 0 END 
#FROM login
#2020-10-12|1
#2020-10-12|1
#2020-10-12|1
#2020-10-13|0
#2020-10-13|0
#2020-10-14|0
#2020-10-14|1
#2020-10-15|0
# 上面是只用CASE 的结果就是一行行 来判断是0 还是1 然后形成新的一列
# 用SUM 的话 我们就是 结合 GROUP BY 因为我们要知道 每个日期总的有几个1 这些1 都是第一次登录

#SELECT date, 
#SUM(CASE
#  WHEN (user_id, date) IN (SELECT user_id, min(date) FROM login GROUP BY user_id)
 #  THEN 1 ELSE 0 END) 
#FROM login
#GROUP BY date
#ORDER BY date ASC
#这个方法的弊端在于如果统一用户在最近一天登录多次那么会重复计数。 虽然可以通过 所有测试例子
# 但是还是要对 用户和日期去重 但是不知道怎么做因为是一行一行的判断。 第一行是用户2 第一次登录
# 在12 号 但是第二行又是用户2 登录在12 号 就没办法了 这种情况我们应该给0 

# 有个想法 可以制造出一张表 保函 唯一的 (user_id, date) 所以我们不用原表LOGIN 去用CASE SUM
# 我们用这个去重的新表上做
SELECT date, 
SUM(CASE 
    WHEN (uni_login.user_id, uni_login.date) 
    IN (SELECT login.user_id, min(login.date) FROM login GROUP BY user_id) 
    THEN 1 ELSE 0 END) FROM 
(SELECT user_id, date FROM login 
GROUP BY user_id, date) AS uni_login
GROUP BY uni_login.date 
ORDER BY uni_login.date ASC


In [ ]:
SELECT date,
SUM(CASE
   WHEN (user_id, date) IN (SELECT user_id, min(date) FROM login GROUP BY user_id)
   THEN 1 ELSE 0 END)
FROM login
GROUP BY date
ORDER BY date ASC

### 5. 牛客每天有很多人登录，请你统计一下牛客每个日期新用户的次日留存率。

In [ ]:
# new customer left rate each day
# log in table: date, user_id 
# so we need to look through all date and verdict the user logged in on this date is the 
# new user this means it has a new user_id and it has not been used before. 
# one table: all distinct user_id
# one table group dates per group and get its new customer 

# new users' first day of login
#SELECT user_id, min(date) FROM login GROUP BY user_id

# these new users stay rate from there first day 
# 这里用LEFT JOIN 应该也是改变了 LOGIN 表格 但是新生成的表格里 因为是左连接 所以 A 表格中全员保留 
# 而LOGIN 中 只 保留了 第二天出现过的user
SELECT a.date, COUNT(DISTINCT login.user_id)/COUNT(a.user_id) as aa FROM
(SELECT user_id, min(date) AS date FROM login GROUP BY user_id) AS a
LEFT JOIN login
ON a.user_id=login.user_id AND login.date=DATE_ADD(a.date,INTERVAL 1 DAY)
GROUP BY a.date

#
SELECT a.date, a.user_id FROM
(SELECT user_id, min(date) AS date FROM login GROUP BY user_id) AS a, login
WHERE a.user_id=login.user_id AND login.date=DATE_ADD(a.date,INTERVAL 1 DAY)
# filter 符合条件的 user 就是是该日期内的新用户 而且第二天登陆了 之余 其他日期内再次出现不考虑

#### 新用户 首次登录日期及其存留情况。NONE 表示没有这样的ID 所以没有存留》 LEFT JOIN

In [1]:
SELECT a.date, a.user_id, login.user_id as aa FROM
(SELECT user_id, min(date) AS date FROM login GROUP BY user_id) AS a
LEFT JOIN login 
ON a.user_id=login.user_id AND login.date=DATE_ADD(a.date,INTERVAL 1 DAY) 

#### 
NONE means login.user_id does not log in at the date of 2020-10-13 
so left join will keep all existed user_id from left table which is a in this case then it will join login table 
based on conditions. the conditions are the user_id from table a and if they also login on next date of current date from
login table. So there is no this user id 3 who log in the first time at 2020-10-12 but also login at 2020-10-13 
2020-10-12|2|2
2020-10-12|3|None
2020-10-12|1|1
2020-10-14|4|4

SyntaxError: invalid syntax (<ipython-input-1-e0e9260da6f7>, line 1)

#### 对比之下 LEFT JOIN 和 WHERE 自联结

#### 新用户首次登录日期并且隔日再登录日期情况 WHERE 自联结

In [ ]:
#### WHERE 
SELECT a.date, a.user_id, login.* FROM
(SELECT user_id, min(date) AS date FROM login GROUP BY user_id) AS a, login
#LEFT JOIN login 
#ON a.user_id=login.user_id AND login.date=DATE_ADD(a.date,INTERVAL 1 DAY) 
WHERE a.user_id=login.user_id AND login.date=DATE_ADD(a.date,INTERVAL 1 DAY)
###
WHERE 做了一个自联结 出现的就是 符合条件的 新用户且第二天登录了。 联结的 LOGIN 部分就是符合条件的部分 没有任何NONE 
2020-10-12|2|4|2|2|2020-10-13
2020-10-12|1|5|1|2|2020-10-13
2020-10-14|4|8|4|1|2020-10-15

** ！！！！ 方法一 UNION + LEFT JOIN. LEFT JOIN WILL KEEP NONE CASES

In [ ]:
#### 存留率计算 基于上面的部分展开
#### 而LOGIN 中 只 保留了 第二天出现过的user
SELECT a.date, ROUND(COUNT(DISTINCT login.user_id) / COUNT(a.user_id),3) AS stay_rate FROM
(SELECT user_id, min(date) AS date FROM login GROUP BY user_id) AS a
LEFT JOIN login 
ON a.user_id=login.user_id AND login.date=DATE_ADD(a.date,INTERVAL 1 DAY) 
GROUP BY a.date
####  UNION 用于纵向 链接表格就是 一行行 来UNION 的
UNION 
SELECT date, 0.000 as p FROM login
WHERE date NOT IN (SELECT min(date) FROM login GROUP BY user_id)
ORDER BY date ASC 

In [ ]:
！！！！ 方法2 用CASE 来判断 然后求商 based on WHERE we use CASE when 

step 1: get the number of new user on each date. 0 means all user are old users 
    SELECT date,

/
SUM(CASE
    WHEN (user_id, date) 
    IN (SELECT user_id, min(date) FROM login GROUP BY user_id)
    THEN 1 ELSE 0 END
   )
FROM login 
GROUP BY date

# get
2020-10-12|3
2020-10-13|0
2020-10-14|1
2020-10-15|0

step 2: get the number of users who stay and log in again at the next day of their first log in date
    SELECT date,

SUM(CASE
    WHEN (user_id, DATE_ADD(date, INTERVAL 1 DAY)) 
    IN (SELECT user_id, date FROM login GROUP BY user_id, date)
    THEN 1 ELSE 0 END
   )
FROM login 
GROUP BY date
ORDER BY date ASC

##get 
2020-10-12|2
2020-10-13|0
2020-10-14|1
2020-10-15|0

step 3:combine 1 and 2 
SELECT date,
ROUND(
SUM(CASE
    WHEN (user_id, DATE_ADD(date, INTERVAL 1 DAY)) 
    IN (SELECT user_id, date FROM login GROUP BY user_id, date)
    THEN 1 ELSE 0 END
   )/
SUM(CASE
    WHEN (user_id, date) 
    IN (SELECT user_id, min(date) FROM login GROUP BY user_id)
    THEN 1 ELSE 0 END
   ),3
    )
FROM login 
GROUP BY date
ORDER BY date ASC

##GET 
2020-10-12|0.667
2020-10-13|None
2020-10-14|1.000
2020-10-15|None
    
STEP 4 IN SQL 0/0 =NONE 
THEN how to cope with NONE and replace then with 0 
ifnull? 

SELECT date,
IFNULL(ROUND(
SUM(CASE
    WHEN (user_id, DATE_ADD(date, INTERVAL 1 DAY)) 
    IN (SELECT user_id, date FROM login GROUP BY user_id, date)
    THEN 1 ELSE 0 END
   )/
SUM(CASE
    WHEN (user_id, date) 
    IN (SELECT user_id, min(date) FROM login GROUP BY user_id)
    THEN 1 ELSE 0 END
   ),3
    ),0
       )
FROM login 
GROUP BY date
ORDER BY date ASC

### 6. 

In [ ]:
# we do not need to show the user who logged in but without doing any tasks 
SELECT p1.user_id, p1.date, p2.number FROM passing_number p1, passing_number p2 
WHERE p1.user_id=p2.user_id and p2.date >= p1.date

#get 
2|2020-10-12|4
3|2020-10-12|1
1|2020-10-13|0
3|2020-10-13|2
3|2020-10-12|2

2|2020-10-12|2020-10-12|4
3|2020-10-12|2020-10-12|1
1|2020-10-13|2020-10-13|0
3|2020-10-13|2020-10-13|2
3|2020-10-12|2020-10-13|2

上面用P2 来判断截止日期之前是错误的




SELECT p1.user_id, p1.date, p2.number FROM passing_number p1, passing_number p2 
WHERE p1.user_id=p2.user_id and p1.date >= p2.date
# get
2|2020-10-12|4
3|2020-10-13|1
3|2020-10-12|1
1|2020-10-13|0
3|2020-10-13|2

2|2020-10-12|2020-10-12|4
3|2020-10-13|2020-10-12|1
3|2020-10-12|2020-10-12|1
1|2020-10-13|2020-10-13|0
3|2020-10-13|2020-10-13|2

为什么结果不一样 当我们用自联结时候 WHERE 在干什么
第一个中我们选的 P1 USERID DATE 它会从 P1 表中 一条一条 拿出来  匹配 WHERE CONDITION
第一条 USERID 2 满足时 有没有  p1.date >= p2.date?  有一条这样的SUM 它对应 NUMBER 是4
当我们到P1 第二条时候 USERID 3， 当前P1.DATE 是2012，满足p1.date >= p2.date 只有一条 就是当前日期截止之前USER 3 只有一条
遍历到USER 1， 满足p1.date >= p2.date 的P2 对应的NUMVBER 是0. 
USER 3， 此时 P1 DATE 2010、10、13 对应p1.date >= p2.date 的P2 DATE 有两条所以 会得到两条记录

3|2020-10-13|2020-10-12|1
3|2020-10-13|2020-10-13|2


#### solution 1 BASED ON ANALYSIS ABOVE

In [ ]:
### SOLUTION 1
SELECT n.name AS u_n, l.date, l.ps_num FROM user n 
INNER JOIN 
(SELECT p1.user_id, p1.date, SUM(p2.number) AS ps_num FROM passing_number p1, passing_number p2 
WHERE p1.user_id=p2.user_id and p1.date >= p2.date
GROUP BY p1.user_id, p1.date) l
ON n.id=l.user_id 
ORDER BY l.date ASC, u_n ASC

#### solution 2 BASED ON ANALYSIS ABOVE

In [ ]:

SELECT user_id, SUM(number) FROM passing_number
GROUP BY user_id 

# get 
2|4
3|3
1|0 
sum all numbers per user_id


2. 
SELECT user_id, SUM(number) OVER(PARTITION BY user_id ORDER BY date) FROM passing_number

# get 
This will keep all user_id with different date from table
1|0
2|4
3|1
3|3

In [ ]:
SELECT u.name as u_n, l_p.date, l_p.number as ps_num 
FROM
(SELECT l.user_id, l.date, p.number FROM login l 
LEFT JOIN passing_number p 
ON l.user_id=p.user_id) AS l_p
LEFT JOIN user u 
ON l_p.user_id=u.id 
WHERE l_p.number >0
ORDER BY l_p.date ASC, u_n ASC

#result
fh|2020-10-12|4
wangchao|2020-10-12|2
wangchao|2020-10-12|1
wangchao|2020-10-13|2
wangchao|2020-10-13|1

# 以及截止到某天，累计总共通过了多少题
# SUM*() OVER*(PARTITION BY XX ORDER BYXX) 实现分区累加 按照哪个分类例如同意姓名的相加
SELECT u.name as u_n, lp.date, 
SUM(lp.number) OVER(partition by lp.user_id ORDER BY date) as ps_num FROM 
(SELECT l.user_id, l.date, p.number FROM login l 
INNER JOIN passing_number p 
ON l.user_id=p.user_id and l.date=p.date) AS lp,
user u
#WHERE lp.date <= pn.date AND lp.user_id=pn.user_id
#GROUP BY lp.user_id, lp.date
WHERE lp.user_id=u.id
ORDER BY lp.date ASC, u.name ASC


In [ ]:
####  总结这题考察的是分组 且对 截止多少多少前求累加值
我还是更倾向第一种方法。 我把 每个用户每个日期 截止前 的 做题数目先求出来。 
再加上对 用户号 登录日期 GROUP BY 用SUM 可以求出 每个用户每个日期截止期总的做题数目

### 第二种的话 用的SUM OVER 他应该是 用窗口函数按姓名分区按日期排序计算累计数
所以 这个按日期排序一定要有 不然 SUM 不知道按照什么来累加。 我们给了 日期排序的话 他会知道按照日期累加 